In [4]:
%useLatestDescriptors
%use coroutines
USE {
  repositories {
    mavenLocal()
    mavenCentral()
  }
  dependencies(
    "dev.mokksy.aimocks:ai-mocks-openai-standalone:0.6.1",
    "io.kotest:kotest-assertions-core:5.9.1",
    "dev.langchain4j:langchain4j-open-ai:1.8.0",
    "dev.langchain4j:langchain4j-kotlin:1.8.0-beta15",
  )
}

In [7]:
import dev.mokksy.aimocks.openai.MockOpenai
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.kotlin.model.chat.chat
import dev.langchain4j.model.openai.OpenAiChatModel
import dev.langchain4j.model.openai.OpenAiChatRequestParameters
import dev.langchain4j.model.output.FinishReason
import io.kotest.matchers.shouldBe
import io.kotest.matchers.shouldNotBe

val openai = MockOpenai(verbose = true, port = 0)

val model: OpenAiChatModel =
  OpenAiChatModel
    .builder()
    .apiKey("foo")
    .baseUrl(openai.baseUrl())
    .build()

openai.completion {
//  temperature(0.42) = 0.42
//  seed = 100500
  model = "4o"
  maxTokens = 120
} responds {
  assistantContent = "Hello"
  finishReason = "stop"
}

runBlocking {

  val result =
    model.chat {
      parameters =
        OpenAiChatRequestParameters
          .builder()
          .maxCompletionTokens(120)
          .temperature(0.42)
          .modelName("4o")
          .seed(100500)
          .build()
      messages += userMessage("Say Hello")
    }

  result.apply {
    finishReason() shouldBe FinishReason.STOP
    tokenUsage() shouldNotBe null
    aiMessage().text() shouldBe "Hello"
  }

  println("OpenAI Response: ${result.aiMessage().text()}")
}

OpenAI Response: Hello
